In [52]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import seaborn as sns 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Add project root to sys.path
project_root = os.path.abspath("../")  # Adjust path if necessary
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify the path
print("Project root added to sys.path:", project_root)

Project root added to sys.path: c:\Users\antoi\Documents\MA3\ML\ML_course\projects\project2\ML_Stroke_Project


In [40]:
from models import *
from utils import *

# Part 0: loading of the data

- Note that the preprocessing has already been done beforehand. You will find more informations regarding that in the report.


In [29]:
#Loading
path_files="..\data\Raw_MissingDataImputed"
df_T1_=pd.read_excel(path_files+"\TiMeS_matrix_mdImputed_allT1.xlsx")
df_T2_=pd.read_excel(path_files+"\TiMeS_matrix_mdImputed_allT2.xlsx")
df_T3_=pd.read_excel(path_files+"\TiMeS_matrix_mdImputed_allT3.xlsx")
df_T4_=pd.read_excel(path_files+"\TiMeS_matrix_mdImputed_allT4.xlsx")

df_T1=df_T1_.copy()
df_T2=df_T2_.copy()
df_T3=df_T3_.copy()
df_T4=df_T4_.copy()

- As some patients joined the study after the first session T1, and some left before T4, we aim there to take this fact in consideration. The goal is to group people from T1 and the ones that joined at T2, and people at T4 with the ones that left after T3. The reason behind that is that the duration between T1-T2 and T3-T4 is small (few weeks) comapred to the duration between T2 and T3 (6 months).


In [17]:
#Grouping T1-T2 vs T3-T4
temp1=pd.concat((df_T1,df_T2[~df_T2["Patient"].isin(df_T1["Patient"])]))
temp2=pd.concat((df_T4,df_T3[~df_T3["Patient"].isin(df_T4["Patient"])]))
list_p = temp1[temp1["Patient"].isin(temp2["Patient"])]["Patient"]
df_T1_T2_=temp1[temp1["Patient"].isin(list_p)].reset_index(drop=True)
df_T3_T4_=temp2[temp2["Patient"].isin(list_p)].reset_index(drop=True)

In [41]:
df_T1_T2=df_T1_T2_.drop(columns="Patient").dropna(axis=1).copy()
df_T3_T4=df_T3_T4_[df_T1_T2.columns]

In [53]:
df_labeled=labelling(df_T3_T4)
df_labeled

0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    0
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    0
34    0
35    1
36    1
37    0
38    1
39    1
40    1
41    1
42    1
43    1
44    0
45    1
46    1
47    1
48    1
49    1
50    1
51    1
52    1
53    1
54    1
55    1
56    0
57    1
58    1
Name: Recovered, dtype: int32

# 1st Part: Clustering approach

In [ ]:
def compute_features_from_connectomes(path_T1, path_T3):
    """
    Computes PCA features based on the relative difference of connectomes between T1 and T3.
    Args:
        path_T1 (str): Directory path for T1 connectomes.
        path_T3 (str): Directory path for T3 connectomes.
    Returns:
        pd.DataFrame: PCA-transformed features with patient IDs.
    """
    # Identify common patients
    common_patients = get_common_patients(path_T1, path_T3)
    
    # Process T1 and T3 connectomes
    connectomes_T1 = process_connectomes(path_T1, common_patients)
    connectomes_T3 = process_connectomes(path_T3, common_patients)

    # Compute relative difference (T1 - T3) / T1
    patient_ids = []
    flattened_matrices = []
    for patient_id in common_patients:
        values_T1 = np.array(connectomes_T1[patient_id])
        values_T3 = np.array(connectomes_T3[patient_id])
        values_T1[values_T1 == 0] = 1  # Avoid division by zero
        relative_difference = (values_T1 - values_T3) / values_T1
        patient_ids.append(patient_id)
        flattened_matrices.append(relative_difference)

    # Normalize the features
    flattened_matrices = np.array(flattened_matrices)
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(flattened_matrices)

    # Apply PCA
    pca = PCA(n_components=0.8)
    pca_features = pd.DataFrame(pca.fit_transform(normalized_features))
    pca_features["Patient"] = patient_ids
    pca_features["Patient"]= pca_features["Patient"].apply(lambda x: "P"+str(x))
    return pca_features


In [ ]:
# Step 0: Get the temporal features, using (T1-T3)/T1,
# for each patient present in both T1 and T3
def get_df_temp(df1, df2, features):
    df_tot = df1[df1.loc[:, "Patient"].isin(df2["Patient"])].reset_index(drop=True)
    A = df1["Patient"].isin(df2["Patient"])
    list_patients = df1.loc[A.values, "Patient"].values
    A = df1[df1["Patient"].isin(list_patients)][features].reset_index(drop=True).copy()
    B = df2[df2["Patient"].isin(list_patients)][features].reset_index(drop=True).copy()
    A_replaced = A.replace(0, 1)
    feature_diff = (A - B) / A_replaced
    df_tot[features] = feature_diff
    return df_tot

In [ ]:
# Step 1: Data Preprocessing, with either min_max and z-standardization, 
def preprocess_temporal_data(df1, df2, df_raw, features, general_features, impute_strategy="mean", normalization=None, use_general_features=True):
    df_tot = get_df_temp(df1, df2, features)
    df_tot = pd.merge(df_tot, df_raw, on="Patient")
    print(features)
    print(general_features[1:])
    if use_general_features:
        all_features = general_features[1:] + features
    else:
        all_features = features

    data_selected = df_tot[["Patient"] + all_features].dropna(axis=0).reset_index(drop=True)
    patients = data_selected["Patient"]
    data_features = data_selected.drop(columns="Patient")
    imputer = SimpleImputer(strategy=impute_strategy)#peut ete pas needed mais pca marche pas avec des NaN's
    data_imputed = pd.DataFrame(imputer.fit_transform(data_features), columns=all_features)

    if normalization == "scaler":
        scaler = StandardScaler()
        data_scaled = pd.DataFrame(scaler.fit_transform(data_imputed), columns=all_features)
    elif normalization == "min_max":
        scaler = MinMaxScaler()
        data_scaled = pd.DataFrame(scaler.fit_transform(data_imputed), columns=all_features)
    else:
        data_scaled = data_imputed

    data_scaled["Patient"] = patients.values
    return data_scaled

In [ ]:
# Step 2: Clustering and output of the metrics for evaluation
def cluster_and_evaluate(data, algorithm="kmeans", params=None):
    if algorithm == "kmeans":
        model = KMeans(n_clusters=params.get("n_clusters", 2), random_state=42)
    elif algorithm == "gmm":
        model = GaussianMixture(
            n_components=params.get("n_clusters", 2),
            covariance_type=params.get("covariance_type", "full"),
            random_state=42
        )
    elif algorithm == "hac":
        model = AgglomerativeClustering(n_clusters=params.get("n_clusters", 2), linkage=params.get("linkage", "ward"))
    else:
        raise ValueError("Unsupported algorithm")

    clusters = model.fit_predict(data)
    silhouette = silhouette_score(data, clusters) if len(set(clusters)) > 1 else None
    dbi = davies_bouldin_score(data, clusters) if len(set(clusters)) > 1 else None
    ch_score = calinski_harabasz_score(data, clusters) if len(set(clusters)) > 1 else None
    
    return {
        "clusters": clusters,
        "silhouette_score": silhouette,
        "dbi": dbi,
        "ch_score": ch_score,
    }

In [ ]:
# Step 3: Visualize Clusters
def visualize_clusters(data, clusters, idx, algorithm, preprocess):
    """
    Visualize clusters using PCA with additional details in the title and labels.
    
    Args:
        data (pd.DataFrame): Data to visualize.
        clusters (np.array): Cluster labels.
        idx (str): Index for the pipeline step.
        algorithm (str): The clustering algorithm used.
        preprocess (str): The preprocessing method used.
    """
    pca = PCA(n_components=2, random_state=42)
    data_reduced = pca.fit_transform(data)
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(data_reduced[:, 0], data_reduced[:, 1], c=clusters, cmap="viridis", s=50, alpha=0.7)
    plt.colorbar(scatter, label="Cluster")
    plt.title(f"PCA Visualization of Clusters\nModel: {algorithm}, Preprocessing: {preprocess}, Step: {idx}")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.tight_layout()
    plt.show()

In [ ]:
# Step 4: Analyze Clusters
def analyze_clusters(data, clusters, features):
    data = data.copy()
    data["Cluster"] = clusters

    # Only aggregate numeric columns
    numeric_features = data[features].select_dtypes(include=[np.number]).columns.tolist()

    summary = data.groupby("Cluster")[numeric_features].agg(["mean", "median", "std"])
    print("\nCluster Summary Statistics:\n", summary)

    # Plot feature distributions
    for feature in numeric_features:
        plt.figure(figsize=(8, 5))
        sns.boxplot(x="Cluster", y=feature, data=data)
        plt.title(f"Feature Distribution by Cluster: {feature}")
        plt.xlabel("Cluster")
        plt.ylabel(feature)
        plt.show()
    return summary

In [ ]:
# Step 5: Main Pipeline
def main_temporal_pipeline(df1, df2, df_general, features, general_features, 
                           algorithms, params, preprocesses, idx, paths_connectomes,
                           use_pca_before_clustering=True,
                           use_general_features=True, use_connectomes_features=True):
    results = []
    for algo in algorithms:
        for preprocess in preprocesses:
            df_temporal_features = get_df_temp(df1, df2, features)
            df_general_filtered = df_general[df_general["Patient"].isin(df_temporal_features["Patient"])].reset_index(drop=True)
            data_temporal_preprocessed = preprocess_temporal_data(
                df1, df2, df_general_filtered, features, general_features, "mean", preprocess, 
                use_general_features=use_general_features,
            )
            if use_connectomes_features==True:
                connectomes_features=compute_features_from_connectomes(paths_connectomes[0],paths_connectomes[1])
                clustering_data = pd.merge(data_temporal_preprocessed, connectomes_features, on="Patient", how="inner")
            clustering_data = data_temporal_preprocessed.drop(columns=["Patient"])
            if use_pca_before_clustering==True:
                pca=PCA(n_components=0.9, random_state=42)
                clustering_data=pca.fit_transform(clustering_data)
            result = cluster_and_evaluate(clustering_data, algorithm=algo, params=params.get(algo, {}))
            print(result)
            if (result["silhouette_score"] is not None and 
                result["dbi"] is not None and 
                result["ch_score"] is not None and 
                result["silhouette_score"] > 0.2 and 
                result["dbi"] < 3 and 
                result["ch_score"] > 3):
                
                if use_general_features:
                    analyzed_features = features + general_features[1:]
                else:
                    analyzed_features = features
                
                print(f"\nTesting {algo} with features: {analyzed_features} and preprocessing {preprocess}")
                visualize_clusters(clustering_data, result["clusters"], idx, algo, preprocess)
                #summary=analyze_clusters(data_temporal_preprocessed, result["clusters"], analyzed_features)
                
                results.append({"features": analyzed_features, "algorithm": algo, "metrics": result})
    
    return results#, summary

In [ ]:
#Algo and preprocessing

algorithms = ["kmeans", "gmm", "hac"]
preprocesses = ["scaler","min_max"]
params = {
    "kmeans": {"n_clusters": 2},
    "gmm": {"n_clusters": 2, "covariance_type": "full"},
    "hac": {"n_clusters": 2, "linkage": "complete"},
}

In [ ]:
#Processing to get only the general features we want 

df_general = pd.get_dummies(df_general[general_features], columns=["Gender","Affected_side"], drop_first=True)
df_general.rename(columns={"Gender_2": "Is_Male", "Affected_side_2": "Is_Right_Side_Affected"}, inplace=True)
general_features = ["Patient", "Age", "Is_Male", "Is_Right_Side_Affected", "Thrombolysis"]
df_general_final = df_general.groupby("Patient").agg("mean").reset_index()

In [ ]:
#Removing collinear features

patients_T1=df_T1["Patient"].values
patients_T3=df_T3["Patient"].values

df_T1_corr=remove_collinear_features(df_T1.drop(columns="Patient"),0.6)
df_T3_corr=remove_collinear_features(df_T3.drop(columns="Patient"),0.6)

df_T1_corr["Patient"]=patients_T1
df_T3_corr["Patient"]=patients_T3

In [ ]:
#Modular approach: possibility to choose the categories of features for the clustering 

use_connectomes_features = True 
use_general_features = True  
use_pca_before_clustering = True
if len(df_T1_corr.columns)>len(df_T3_corr.columns):
    features=df_T3_corr.drop(columns="Patient").columns.tolist()
else:
    features=df_T1_corr.drop(columns="Patient").columns.tolist()

#features=motor_features+attention_features+executive_features+memory_features+sensory_features+language_features+neglect_features

results = main_temporal_pipeline(df_T1, df_T3, df_general_final, features, general_features, 
                                 algorithms, params, preprocesses, "T3-T1", 
                                 paths_connectomes, use_pca_before_clustering=use_pca_before_clustering,
                                 use_general_features=use_general_features,
                                 use_connectomes_features=use_connectomes_features)



# 2st Part: Regression and classification approach

- The aim of this part is to first use a multi target regression to predict the T4 features from T1. We will add the patients from T2 and T3 as explained before, to get more data. Multiple models are trained and we compare their metrics. Multiples way of processing the data are also proposed.

In [ ]:
# Feature sets

# Standarsized features
X_std=Processing_std(df_T1_T2)
Y_std=Processing_std(df_T3_T4)

#Features that show the less collinearity
X_corr=Processing_collinear(df_T1_T2)

# Features after a Random forest with a treshold set at 0.02
X_selected_rf, _ = Processing_rf_features(df_T1_T2,df_T3_T4)

# Features after PCA reduction
X_pca, explained_variance_X = Processing_PCA(df_T1_T2, n_components=0.8)

# Features after Partial Least Squares
X_std = Processing_std(df_T1_T2)  # Standardized features
Y_std = Processing_std(df_T3_T4)  # Standardized targets

Y_pls, pls_model = Processing_PLS(X_std, Y_std, n_components=5)


feature_sets = {
    #'AllFeatures': X_std,
    'SelectedFeatures_RF': X_selected_rf,
    'Reduced_X_PCA': X_pca,
    'Non_collinear_features': X_corr
}

# Targets
target_sets = {
    'Original_Y': Y_std,
    'Features_PLS': Y_pls
}

# Models
models = {
    'random_forest_model_mo' : random_forest_multi_out_model,
    #'MSVR': msvr_model,
    'RidgeRegression': ridge_multi_out_model,
    'GradientBoosting': gradient_boost_multi_out_model
}

In [ ]:
results = []

for feature_name, X_features in feature_sets.items():
    for target_name, Y_targets in target_sets.items():
        for model_name, model_func in models.items():
            print(f"Evaluating {model_name} with {feature_name} and {target_name}...")
            metrics,_ = model_func(X_features, Y_targets)  # Cross-validation is applied inside
            print("current metrics",metrics)
            results.append({
                'Model': model_name,
                'Features': feature_name,
                'Targets': target_name,
                'MAE': metrics['MAE'],
                'RMSE': metrics['RMSE']
            })

# Convert results to DataFrame for better readability

results_df = pd.DataFrame(results)
print(results_df.sort_values(by='RMSE'))


Evaluating random_forest_model_mo with SelectedFeatures_RF and Original_Y...
current metrics {'MAE': 0.7770820425075375, 'RMSE': 1.0983902299626476}
Evaluating RidgeRegression with SelectedFeatures_RF and Original_Y...
current metrics {'MAE': 0.8205712905943764, 'RMSE': 1.1449219946597657}
Evaluating GradientBoosting with SelectedFeatures_RF and Original_Y...
current metrics {'MAE': 0.7421691982814121, 'RMSE': 1.066791968930892}
Evaluating random_forest_model_mo with SelectedFeatures_RF and Features_PLS...
current metrics {'MAE': 2.0807327232940187, 'RMSE': 2.7349962108086445}
Evaluating RidgeRegression with SelectedFeatures_RF and Features_PLS...
current metrics {'MAE': 2.1364205700664884, 'RMSE': 2.855096722419211}
Evaluating GradientBoosting with SelectedFeatures_RF and Features_PLS...
current metrics {'MAE': 1.9760628978018708, 'RMSE': 2.5983205794409896}
Evaluating random_forest_model_mo with Reduced_X_PCA and Original_Y...
current metrics {'MAE': 0.7639917316774779, 'RMSE': 1.086

- THe ridge and gradient boosting regressions seem to have the best performances, when combined with the Non_collinear_features and the X features after PCA reduction. 

- Now we can start classifying

In [ ]:
#Target
y = labelling(df_T3_T4) 

# Preprocess data
X_std = Processing_std(df_T1_T2)

# Remove correlated features and standardize
X_corr = Processing_collinear(df_T1_T2)

# Handle imbalance
X_balanced, y_balanced = Processing_imbalance(X_std, y)
X_balanced_corr, y_balanced_corr = Processing_imbalance(X_corr, y)

# Feature sets
feature_sets = {
    'Standardized_Features': X_std,
    'Non_collinear_features': X_corr,
    'Balanced_Data': X_balanced
}

# Models
models = {
    'LogisticRegression': logistic_regression_model,
    'RandomForest': random_forest_model,
    'GradientBoosting': gradient_boosting_model
}

0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    0
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    0
34    0
35    1
36    1
37    0
38    1
39    1
40    1
41    1
42    1
43    1
44    0
45    1
46    1
47    1
48    1
49    1
50    1
51    1
52    1
53    1
54    1
55    1
56    0
57    1
58    1
Name: Recovered, dtype: int32

In [ ]:
# Evaluate combinations
results = []
for feature_name, X_features in feature_sets.items():
    for model_name, model_func in models.items():
        print(f"Evaluating {model_name} with {feature_name}...")
        metrics, _ = model_func(X_features, y_balanced)
        results.append({
            'Model': model_name,
            'Features': feature_name,
            'Accuracy': metrics['Accuracy'],
            'Precision': metrics['Precision'],
            'Recall': metrics['Recall'],
            'F1-Score': metrics['F1-Score']
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='F1-Score', ascending=False))
